<a href="https://colab.research.google.com/github/Aarush49/RAG-Project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Dependencies

In [ ]:
!pip install chromadb sentence-transformers pypdf google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.7 MB/s eta

# Step 2: Upload and Extract text from PDF

In [ ]:
from google.colab import files
from pypdf import PdfReader

uploaded = files.upload()
pdf_path = next(iter(uploaded))
reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
  # TODO: Extract text from the page and add it to the 'text' variable
  text += page.extract_text() + "\n"

print(text[:1000])  # Preview first 1000 chars

Saving Project 1 Pseudocode.pdf to Project 1 Pseudocode.pdf
Main.cpp  
Main  
Returns:  int  Parameters:  None  1.  Prompt  user  for  input  filename  2.  Open  input  file  3.  If  file  is  not  open  1.  Return  -1  4.  Parse  input  file  and  store  player  data  in  linked  list  5.  Close  input  file  6.  Call  LinkedList  sort  function  on  players  LinkedList  7.  Set  max  decimal  points  to  3  8.  Print  Players  9.  Find  and  print  league  leaders  10.  Return  0  
Parse  Input  File  
Returns:  LinkedList  with  player  information  Parameters:  Input  FIle  1.  Initialize  LinkedList  2.  While  there  is  a  new  line  in  input  file  1.  Read  player  name  2.  If  player  with  the  same  name  already  exists  in  linked  list  1.  Retrieve  already  existing  player  object  3.  Else  1.  Initialize  new  player  object  and  add  player  to  linked  list  4.  For  each  letter  in  the  input  batting  records  1.  Switch  batting  record  1.  Case  H  a.  I

# Step 3: Chunk and Vectorize Text

In [ ]:
from sentence_transformers import SentenceTransformer

CHUNK_SIZE = 500
chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
embedder = SentenceTransformer('all-MiniLM-L6-v2')
# TODO: Generate embeddings for the chunks
embeddings = embedder.encode(chunks)
print(embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 4: Use ChromaDB for storing vectorized text

In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.create_collection(name="pdf_notes")
ids = [f"chunk_{i}" for i in range(len(chunks))]
# TODO: Add the documents, embeddings, and ids to the collection
collection.add(documents = chunks, embeddings = embeddings.tolist(), ids = ids)


# Step 5: Take input and generate context

In [ ]:
query = input("Enter your question: ")
query_emb = embedder.encode([query])[0]

results = collection.query(query_embeddings=[query_emb], n_results=3)

context = "\n---\n".join(results['documents'][0])
print("Context:\n", context)

Enter your question: what does the main function do in the program?
Context:
 Main.cpp  
Main  
Returns:  int  Parameters:  None  1.  Prompt  user  for  input  filename  2.  Open  input  file  3.  If  file  is  not  open  1.  Return  -1  4.  Parse  input  file  and  store  player  data  in  linked  list  5.  Close  input  file  6.  Call  LinkedList  sort  function  on  players  LinkedList  7.  Set  max  decimal  points  to  3  8.  Print  Players  9.  Find  and  print  league  leaders  10.  Return  0  
Parse  Input  File  
Returns:  LinkedList  with  player  information  P
---
  strikeouts,  hits  by  pitch,  sacrifices,  
batting
 
average
 
(to
 
3
 
decimal
 
places),
 
and
 
on
 
base
 
percentage
 
(to
 
3
 
decimal
 
places),
 
with
 
a
 
tab
 
between
 
each
 
field
 2.  Print  new  line  
Find  and  Print  Leaders  
Returns:  Nothing  Parameters:  Reference  to  LinkedList  with  player  information  1.  Initialize  variables  to  store  best  value  for  each  category  (battin

# Step 6: Use Gemini to answer questions from the context

In [ ]:
from google.colab import userdata
import google.generativeai as genai

# Make sure to have API_KEY in your secrets
genai.configure(api_key=userdata.get("API_KEY"))

model = genai.GenerativeModel('models/gemini-2.5-flash')

prompt = f"Answer the follwoing question using the context given. Context: {context} \n Question: {query}"

response = model.generate_content(prompt)
print("Answer:", response.text)

Answer: The main function in the program performs the following actions:

1.  Prompts the user for an input filename.
2.  Opens the input file.
3.  Checks if the file failed to open; if so, it returns -1.
4.  Parses the input file to extract player data and stores it in a linked list.
5.  Closes the input file.
6.  Calls a sort function on the players linked list.
7.  Sets the maximum decimal points for output to 3.
8.  Prints the player data.
9.  Finds and prints the league leaders.
10. Returns 0 upon completion.
